In [8]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# Database connection parameters
DB_HOST = "localhost"       
DB_USER = "root"           
DB_PASSWORD = "chag" 
DB_NAME = "sakila"          

engine = create_engine(f"mysql+mysqlconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}")

def rentals_month(engine, month, year):

    query = f"""
        SELECT rental_id, customer_id, rental_date
        FROM rental
        WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year};
    """
    df = pd.read_sql(query, engine)
    return df

def rental_count_month(df_rentals, month, year):
    """
    Counts the number of rentals per customer for a given month and year.

    Arguments:
    - df_rentals : DataFrame of rentals for a given month.
    - month : month (e.g., 5 for May)
    - year : year (e.g., 2005)

    Returns:
    - A DataFrame containing the number of rentals per customer.
    """
    df_count = df_rentals.groupby("customer_id")["rental_id"].count().reset_index()
    df_count.columns = ["customer_id", f"rentals_{month}_{year}"]
    return df_count

def compare_rentals(df1, df2, month1, year1, month2, year2):
    """
    Compares rental counts between two months.

    Arguments:
    - df1 : DataFrame of rentals for the first month.
    - df2 : DataFrame of rentals for the second month.
    - month1, year1 : first month and year (e.g., 5, 2005)
    - month2, year2 : second month and year (e.g., 6, 2005)

    Returns:
    - A merged DataFrame with a 'difference' column indicating the rental change between months.
    """
    df_merged = pd.merge(df1, df2, on="customer_id", how="outer").fillna(0)
    df_merged["difference"] = df_merged[f"rentals_{month2}_{year2}"] - df_merged[f"rentals_{month1}_{year1}"]
    return df_merged

# Get rental data for May and June 2005
df_may = rentals_month(engine, 5, 2005)
df_june = rentals_month(engine, 6, 2005)

# Count rentals per customer for May and June
df_may_count = rental_count_month(df_may, 5, 2005)
df_june_count = rental_count_month(df_june, 6, 2005)

# Compare rental counts between May and June
df_comparison = compare_rentals(df_may_count, df_june_count, 5, 2005, 6, 2005)

df_comparison.head()


,customer_id,rentals_5_2005,rentals_6_2005,difference
0,1,2.0,7.0,5.0
1,2,1.0,1.0,0.0
2,3,2.0,4.0,2.0
3,4,0.0,6.0,6.0
4,5,3.0,5.0,2.0
